In [81]:
import quinn
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql import SparkSession
from datetime import date
from utils import dashes_to_underscores

spark = SparkSession.builder.getOrCreate()

dataset_df = spark.read.csv('../data/Monkey_Pox_Cases_Worldwide.csv', header=True)
daily_df = spark.read.csv('../data/Daily_Country_Wise_Confirmed_Cases.csv', header=True)


total_df = dataset_df.select(dataset_df['Country'], dataset_df['Confirmed_Cases'])

dataset_df = total_df.join(daily_df,['Country'],how='left')

dataset_df = dataset_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df = daily_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df.groupBy("Country").pivot("Country")

pandas_DF = dataset_df.toPandas()

pandas_DF.set_index('Country', inplace=True)
transpose_df = pandas_DF.transpose()
transpose_df.rename(columns = {'Country':'Date'})

transpose_df.reset_index(inplace=True)

spark_df=spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

spark_df.show()

+----------+-------+--------+-----+-------------+-------+-------+------+-----+------+------+---------+-----------+------+-----------+-------+-------+--------+--------+--------------------+--------------+----------------+-----+-------+---------+-------+------+-----+------+-------+---------+-------+------+------+------+-----+------+---------+-------+-----+-------+-------+----------+-----+------+---------+-------+------------+-----------+--------+-------+--------+------+----+-------+-----------+------+
|      Date|England|Portugal|Spain|United States|Germany|Belgium|Sweden|Italy|Canada|France|Australia|Netherlands|Israel|Switzerland|Austria|Denmark|Scotland|Slovenia|United Arab Emirates|Czech Republic|Northern Ireland|Wales|Ireland|Argentina|Finland|Mexico|Malta|Norway|Hungary|Gibraltar|Morocco|Latvia|Greece|Brazil|Ghana|Poland|Venezuela|Romania|Benin|Iceland|Georgia|Luxembourg|Chile|Serbia|Singapore|Lebanon|South Africa|South Korea|Colombia|Croatia|Bulgaria|Taiwan|Peru|Estonia|Puerto Ric

In [2]:
from utils import date_check, date_minus_weeks
import pyspark.sql.functions as sf

def get_countries():
    columns = spark_df.columns
    countries_list = []
    for column in columns:
        if column != "Date":
            countries_list.append(column)
    # for country in countries_list:
    #     print(country)
    return countries_list
get_countries()

def select_all_countries():
    i = 0
    countries = get_countries()
    for country in countries:
        spark_df.select('Date', country).where(spark_df[f"{country}"] >= 1).show()
select_all_countries()

# spark_df.select('Date','United States').where(spark_df['United States'] >= 1).show()
# spark_df.select('Date','United States').where(spark_df.collect(). >= 1).show()

# spark_df.select('*').show()

+----------+-------+
|      Date|England|
+----------+-------+
|2022_05_06|      1|
|2022_05_12|      1|
|2022_05_13|      1|
|2022_05_15|      4|
|2022_05_18|      2|
|2022_05_20|     11|
|2022_05_23|     36|
|2022_05_24|     14|
|2022_05_25|      7|
|2022_05_26|     24|
|2022_05_30|     71|
|2022_05_31|     11|
|2022_06_01|      5|
|2022_06_02|     11|
|2022_06_03|     15|
|2022_06_06|     73|
|2022_06_07|     18|
|2022_06_09|     43|
|2022_06_12|    104|
|2022_06_14|     52|
+----------+-------+
only showing top 20 rows

+----------+--------+
|      Date|Portugal|
+----------+--------+
|2022_05_17|       3|
|2022_05_18|      11|
|2022_05_19|       9|
|2022_05_23|      14|
|2022_05_24|       2|
|2022_05_25|      10|
|2022_05_26|       9|
|2022_05_27|      16|
|2022_05_30|      22|
|2022_05_31|       4|
|2022_06_01|      19|
|2022_06_02|      19|
|2022_06_03|       5|
|2022_06_06|      10|
|2022_06_07|      13|
|2022_06_08|      25|
|2022_06_09|      18|
|2022_06_14|      22|
|2022_06

In [86]:
import pandas as pd
pd_df = spark_df.toPandas()
# pd_df.head(5)
print(pd_df.columns)
print(len(pd_df.columns))

# def get_country_cols(dataframe):
#     # col_list = []
#     for col in dataframe.columns:
#         if col != "Date":
#             # col_list.append(col)
#             # dataframe = dataframe.astype({col: "int64"})
#             print(col)
# get_country_cols(pd_df)
# pd_df.apply(pd.to_numeric(get_country_cols(pd_df)))
# pd_df.apply(pd.to_numeric(pd_df.England))
# pd_df.apply(pd_df.astype(get_country_cols(pd_df)))
pd_df = pd_df.astype({"Date": "string"})
pd_df = pd_df.astype({"England": "int64"})
pd_df = pd_df.astype({"Portugal": "int64"})
pd_df = pd_df.astype({"Spain": "int64"})
pd_df = pd_df.astype({"United States": "int64"})
pd_df = pd_df.astype({"Germany": "int64"})
pd_df = pd_df.astype({"Belgium": "int64"})
pd_df = pd_df.astype({"Sweden": "int64"})
pd_df = pd_df.astype({"Italy": "int64"})
pd_df = pd_df.astype({"Canada": "int64"})
pd_df = pd_df.astype({"France": "int64"})
pd_df = pd_df.astype({"Australia": "int64"})
pd_df = pd_df.astype({"Netherlands": "int64"})
pd_df = pd_df.astype({"Israel": "int64"})
pd_df = pd_df.astype({"Switzerland": "int64"})
pd_df = pd_df.astype({"Austria": "int64"})
pd_df = pd_df.astype({"Denmark": "int64"})
pd_df = pd_df.astype({"Portugal": "int64"})
pd_df = pd_df.astype({"Scotland": "int64"})
pd_df = pd_df.astype({"Slovenia": "int64"})
pd_df = pd_df.astype({"United Arab Emirates": "int64"})
pd_df = pd_df.astype({"Czech Republic": "int64"})
pd_df = pd_df.astype({"Northern Ireland": "int64"})
pd_df = pd_df.astype({"Wales": "int64"})
pd_df = pd_df.astype({"Ireland": "int64"})
pd_df = pd_df.astype({"Argentina": "int64"})
pd_df = pd_df.astype({"Finland": "int64"})
pd_df = pd_df.astype({"Mexico": "int64"})
pd_df = pd_df.astype({"Malta": "int64"})
pd_df = pd_df.astype({"Norway": "int64"})
pd_df = pd_df.astype({"Hungary": "int64"})
pd_df = pd_df.astype({"Gibraltar": "int64"})
pd_df = pd_df.astype({"Morocco": "int64"})
pd_df = pd_df.astype({"Latvia": "int64"})
pd_df = pd_df.astype({"Greece": "int64"})
pd_df = pd_df.astype({"Brazil": "int64"})
pd_df = pd_df.astype({"Ghana": "int64"})
pd_df = pd_df.astype({"Poland": "int64"})
pd_df = pd_df.astype({"Venezuela": "int64"})
pd_df = pd_df.astype({"Romania": "int64"})
pd_df = pd_df.astype({"Benin": "int64"})
pd_df = pd_df.astype({"Iceland": "int64"})
pd_df = pd_df.astype({"Georgia": "int64"})
pd_df = pd_df.astype({"Luxembourg": "int64"})
pd_df = pd_df.astype({"Chile": "int64"})
pd_df = pd_df.astype({"Serbia": "int64"})
pd_df = pd_df.astype({"Singapore": "int64"})
pd_df = pd_df.astype({"Lebanon": "int64"})
pd_df = pd_df.astype({"South Africa": "int64"})
pd_df = pd_df.astype({"South Korea": "int64"})
pd_df = pd_df.astype({"Colombia": "int64"})
pd_df = pd_df.astype({"Croatia": "int64"})
pd_df = pd_df.astype({"Bulgaria": "int64"})
pd_df = pd_df.astype({"Taiwan": "int64"})
pd_df = pd_df.astype({"Peru": "int64"})
pd_df = pd_df.astype({"Estonia": "int64"})
pd_df = pd_df.astype({"Puerto Rico": "int64"})
pd_df = pd_df.astype({"Turkey": "int64"})

pd_df.dtypes
# pd_df.head(5)


Index(['Date', 'England', 'Portugal', 'Spain', 'United States', 'Germany',
       'Belgium', 'Sweden', 'Italy', 'Canada', 'France', 'Australia',
       'Netherlands', 'Israel', 'Switzerland', 'Austria', 'Denmark',
       'Scotland', 'Slovenia', 'United Arab Emirates', 'Czech Republic',
       'Northern Ireland', 'Wales', 'Ireland', 'Argentina', 'Finland',
       'Mexico', 'Malta', 'Norway', 'Hungary', 'Gibraltar', 'Morocco',
       'Latvia', 'Greece', 'Brazil', 'Ghana', 'Poland', 'Venezuela', 'Romania',
       'Benin', 'Iceland', 'Georgia', 'Luxembourg', 'Chile', 'Serbia',
       'Singapore', 'Lebanon', 'South Africa', 'South Korea', 'Colombia',
       'Croatia', 'Bulgaria', 'Taiwan', 'Peru', 'Estonia', 'Puerto Rico',
       'Turkey'],
      dtype='object')
57


Date                    string
England                  int64
Portugal                 int64
Spain                    int64
United States            int64
Germany                  int64
Belgium                  int64
Sweden                   int64
Italy                    int64
Canada                   int64
France                   int64
Australia                int64
Netherlands              int64
Israel                   int64
Switzerland              int64
Austria                  int64
Denmark                  int64
Scotland                 int64
Slovenia                 int64
United Arab Emirates     int64
Czech Republic           int64
Northern Ireland         int64
Wales                    int64
Ireland                  int64
Argentina                int64
Finland                  int64
Mexico                   int64
Malta                    int64
Norway                   int64
Hungary                  int64
Gibraltar                int64
Morocco                  int64
Latvia  

In [88]:
import bar_chart_race as bcr
from ffmpeg import *
from matplotlib import animation

animation.writers.list()

bcr.bar_chart_race(
    df=pd_df,
    title="Monkey_Pox_Cases")

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got linear